In [2]:
#import libraries for API Calls / Data Analysis
import uuid,datetime,requests,pprint,math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
#set Client Environment (from url), Aladdin Username & Password, and APIkey here
client_env = 'santander'

username = 'XXXXXXXX'
pwd = 'XXXXXXXX'
APIkey= 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [7]:
today = datetime.date.today().strftime('%Y-%m-%d')

In [8]:
#function to generate unique headers when called
def generate_headers():
    headers = {
        'VND.com.blackrock.API-Key': APIkey,
        'VND.com.blackrock.Origin-Timestamp': str(datetime.datetime.utcnow().replace(microsecond=0).astimezone().isoformat()),
        'VND.com.blackrock.Request-ID': str(uuid.uuid1())
    }
    return headers

In [9]:
def get_aladdin_securities(assets_list):
    """Get Aladdin identifiers function"""
    #Detalle derivados
    url1 = 'https://santander.blackrock.com/api/reference-data/securities/v1/security-master'
    
    each = 100
    info_table = pd.DataFrame()
    
    for n in np.arange(0,len(assets_list), each):
    
        payload = {
        'assetId': ','.join(assets_list[n:n+each]),
        'assetIdType': 'ISIN',
        }
    
        r = requests.get(url1, auth = (username, pwd), params = payload, headers = generate_headers()).json()
    
        securities_list = []
        #loop through Securities results
        for asset, value in r['assetByAssetId'].items():
            sec_type = list(value.keys())[1]
            securities_list.append(value[sec_type])
    
        #convert to DataFrame
        df_aux = pd.json_normalize(securities_list)
    
        info_table = pd.concat([info_table, df_aux], axis=0, ignore_index=True)
    return info_table

In [ ]:
#LU2843778262 ISIN
assets_list = ['ES0L02502075']
asset_type = 'ISIN'
securities = get_aladdin_securities(assets_list)


In [10]:
df = pd.read_csv(
    r"C:\Users\n740789\Documents\Projects_local\DataSets\DATAFEED\raw_dataset\2025\20250301_Production\20250301_Equities_feed_new_strategies_filtered_old_names_iso_permId.csv",
    usecols=[
            "isin",
            "instrument_type",
            "issuer_name",
            "issuer_country",
            "permId"],
             
    dtype={
            "isin": str,
            "permId": str
        }
    )   

In [11]:
df.rename(columns={"permId": "permid"}, inplace=True)

In [5]:
#get crossreference
crossreference = pd.read_csv(r"C:\Users\n740789\Documents\Projects_local\DataSets\crossreference\Aladdin_Clarity_Issuers_20250301.csv", 
                             dtype={"CLARITY_AI": str})

crossreference.columns = ["brs_id", "brs_issuer_name", "permid", "msci_id", "sustainalytics_id"]

In [12]:
# add "brs_id" to df from crossreference
df = pd.merge(df, crossreference[["brs_id", "permid"]], on="permid", how="left")

In [4]:
for col in df.columns[:10]:
    print(col)

isin
instrument_type
issuer_name
issuer_country
gics2_industry
region
company_inheriting
parent_company
esg_score
esg_score_relevance


In [13]:
df.head()

,isin,instrument_type,issuer_name,issuer_country,permid,brs_id
0,US0527M0W110,FIXED_INCOME,Autobahn Funding Company LLC,United States of America,5000594216,C16979
1,US69372AV254,FIXED_INCOME,PACCAR Financial Corp,United States of America,4295900643,693717
2,US74533AGH59,FIXED_INCOME,Puget Sound Energy Inc,United States of America,5000054387,745332
3,US74533AGQ58,FIXED_INCOME,Puget Sound Energy Inc,United States of America,5000054387,745332
4,XS1545823632,FIXED_INCOME,Morgan Stanley & Co International PLC,United Kingdom of Great Britain and Northern I...,5000076412,R84910
